<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Patch_of_the_New_York_City_Police_Department.svg/800px-Patch_of_the_New_York_City_Police_Department.svg.png" title="NYPD Logo" style="height:7em;float:right"/>
<h1 style="float:left;font-size:2.5em">NYC Motor Vehicle Crashes</h1>

The **goal** of this notebook is to help prepare a dataset to draw insights from regarding high-risk locations for vehicle crashes in New York City.

The **source** dataset is the [Motor Vehicle Collisions crash table](https://data.cityofnewyork.us/d/h9gi-nx95), and originates from [police reports (MV-104AN)](https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/ny_overlay_mv-104an_rev05_2004.pdf) filled in by the [NYPD](https://en.wikipedia.org/wiki/NYPD "New York City Police Department"). It is frequently updated and publicly available. To limit the initial scope, we will use a `.csv` file containing a subset of this data consisting of 100000 rows.

The **target** dataset is a `.csv` file, cleaned as good as possible within the given time constraints, while adhering to the requirements in the checklist below.

**Checklist**
* [ ] No missing values
* [ ] No duplicates
* [ ] Values are consolidated
* [ ] Correct data format
* [ ] No blank spaces


## Background information

Based on: [dataset description](https://data.cityofnewyork.us/d/h9gi-nx95) *(consulted 09 March 2021)*

**Key dataset information:**

* Currently, there are 1.76M rows.
* The table has 29 columns.
* Each row represents a Motor Vehicle Collision.

**Data collection timeline:**

|Period|Description|
|---|:--|
|**Pre 1998**|No uniform way to collect traffic safety data within the NYPD.|
|**April 1998**|NYPD implements the TrafficStat program and creates form `MV-104AN` to help collect data.|
|**July 1999**|NYPD implements Traffic Accident Management System (TAMS) and uses form `MV-104AN`. This marks the start of the collection of basic data to get the number of accidents, injuries and fatalities.|
|**2014**|Start of the VisionZero program to replace TrafficStat.|
|**March 2016**|TAMS is replaced by Finest Online Records Management System (FORMS). FORMS brings electronic input, a data warehouse backend and the collection of extra information with it. An updated version of form `MV-104AN` is being used.|

**Other notes:**

* There is a detailed [data dictionary](https://data.cityofnewyork.us/api/views/h9gi-nx95/files/bd7ab0b2-d48c-48c4-a0a5-590d31a3e120?download=true&filename=MVCollisionsDataDictionary_20190813_ERD.xlsx) document available.
  * Column `collision_id` is a [primary key](https://en.wikipedia.org/wiki/Primary_key).
  * Besides the `Crash` dataset, there are also `Vehicle` and `Person` datasets, where we can use `collision_id` to look up rows.
  * Columns `latitude` and `longitude` are coordinates for [WGS 84](https://en.wikipedia.org/wiki/World_Geodetic_System) (a.k.a. EPSG 4326).

## Preparation

### a) Verification of the software requirements

Make sure you have the following installed or follow the links for installation instructions:
1. `numpy` ([homepage](https://www.numpy.org/) | [installation instructions](https://numpy.org/install/))
1. `pandas` ([homepage](https://pandas.pydata.org/) | [installation instructions](https://pandas.pydata.org/docs/getting_started/install.html))
1. `pandas_profiling` ([github page](https://github.com/pandas-profiling/pandas-profiling) | [installation instructions](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/pages/installation.html))

**Important:** *It's possible you need to activate the widgets extension for this notebook to work properly. See the [Jupyter section](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/pages/installation.html#jupyter-notebook-lab) in the pandas_profiling installation instructions.*

### b) Importing the necessary modules

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

### c) Importing the dataset

#### Standard import

In [2]:
df = pd.read_csv("./data/data_100000.csv")

#### Inspection

In [3]:
df.shape

(100000, 29)

In [4]:
df.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'location', 'on_street_name', 'off_street_name',
       'cross_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5'],
      dtype='object')

In [6]:
# is collision_id a unique value?
df.collision_id.nunique() == len(df)

True

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   crash_date                     100000 non-null  object 
 1   crash_time                     100000 non-null  object 
 2   borough                        64974 non-null   object 
 3   zip_code                       64966 non-null   float64
 4   latitude                       91965 non-null   float64
 5   longitude                      91965 non-null   float64
 6   location                       91965 non-null   object 
 7   on_street_name                 73991 non-null   object 
 8   off_street_name                47125 non-null   object 
 9   cross_street_name              25967 non-null   object 
 10  number_of_persons_injured      100000 non-null  int64  
 11  number_of_persons_killed       100000 non-null  int64  
 12  number_of_pedestrians_injured  

In [8]:
df.head().T

,0,1,2,3,4
crash_date,2017-04-18T00:00:00.000,2017-05-06T00:00:00.000,2017-04-27T00:00:00.000,2017-05-09T00:00:00.000,2017-04-18T00:00:00.000
crash_time,23:10,13:00,17:15,20:10,14:00
borough,STATEN ISLAND,BRONX,QUEENS,NaN,BRONX
zip_code,10312.0,10472.0,11420.0,NaN,10456.0
latitude,40.536728,40.829052,40.677303,40.624958,40.828846
longitude,-74.193344,-73.85038,-73.804565,-74.145775,-73.90312
location,"(40.536728, -74.193344)","(40.829052, -73.85038)","(40.677303, -73.804565)","(40.624958, -74.145775)","(40.828846, -73.90312)"
on_street_name,NaN,CASTLE HILL AVENUE,135 STREET,FOREST AVENUE,NaN
off_street_name,NaN,BLACKROCK AVENUE,FOCH BOULEVARD,RICHMOND AVENUE,NaN
cross_street_name,243 DARLINGTON AVENUE,NaN,NaN,NaN,1167 BOSTON ROAD


> **Notes:** 
> * `crash_date` and `crash_time` *=> can be merged and converted to a datetime datatype*
> * `zip_code` is a float *=> change to string; you can't perform calculations with zip codes*
> * `latitude` and `longitude` columns are duplicated in `location` *=> drop one of them*
> * `collision_id` is a primary key *=> can become the index*
> * `vehicle_type_code1` and `vehicle_type_code2` column names don't have an underscore in their suffix
> * multiple `contributing_factor_vehicle` and `vehicle_type_code` columns with a numeric suffix *=> better to change*
> * there are lots of missing values (NaN), mainly in `contributing_factor_vehicle` and `vehicle_type_code` *=> drop these columns*

## Data preprocessing

### a) Improve dataframe

> **Notes:** 
> * Let's reimport the dataframe like this:
> * `collision_id` as the index
> * `crash_date` and `crash_time` as a datetime
> * `zip_code` as a string
> * other columns: 'borough', 'zip_code', 'latitude', 'longitude', 'location', 'on_street_name', 'off_street_name', 'cross_street_name', 'number_of_persons_injured', 'number_of_persons_killed'

In [9]:
df = pd.read_csv("./data/data_100000.csv", 
                 parse_dates={"timestamp" : ["crash_date", "crash_time"]}, 
                 infer_datetime_format=True, 
                 index_col="collision_id", 
                 dtype={"zip_code" : "str"},
                 usecols=["collision_id", 
                          "crash_date", 
                          "crash_time", 
                          "latitude", 
                          "longitude", 
                          "borough", 
                          "zip_code", 
                          "on_street_name", 
                          "off_street_name", 
                          "cross_street_name", 
                          "number_of_persons_injured", 
                          "number_of_persons_killed"])

In [10]:
column_labels = {'number_of_persons_injured':'injuries', 
                 'number_of_persons_killed':'fatalities'}

df.rename(columns=column_labels, inplace=True)

### b) Reinspection

#### Check a couple of rows

In [13]:
df.head()

,timestamp,borough,zip_code,latitude,longitude,on_street_name,off_street_name,cross_street_name,injuries,fatalities
collision_id,,,,,,,,,,
3654181,2017-04-18 23:10:00,STATEN ISLAND,10312,40.536728,-74.193344,NaN,NaN,243 DARLINGTON AVENUE,0,0
3665311,2017-05-06 13:00:00,BRONX,10472,40.829052,-73.850380,CASTLE HILL AVENUE,BLACKROCK AVENUE,NaN,1,0
3658491,2017-04-27 17:15:00,QUEENS,11420,40.677303,-73.804565,135 STREET,FOCH BOULEVARD,NaN,0,0
3666554,2017-05-09 20:10:00,NaN,NaN,40.624958,-74.145775,FOREST AVENUE,RICHMOND AVENUE,NaN,1,0
3653269,2017-04-18 14:00:00,BRONX,10456,40.828846,-73.903120,NaN,NaN,1167 BOSTON ROAD,0,0


#### Check the data types of the columns

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 3654181 to 4247517
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   timestamp          100000 non-null  datetime64[ns]
 1   borough            64974 non-null   object        
 2   zip_code           64966 non-null   object        
 3   latitude           91965 non-null   float64       
 4   longitude          91965 non-null   float64       
 5   on_street_name     73991 non-null   object        
 6   off_street_name    47125 non-null   object        
 7   cross_street_name  25967 non-null   object        
 8   injuries           100000 non-null  int64         
 9   fatalities         100000 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 8.4+ MB


### c) Create subsets of the data to work on

In [28]:
"""# list the location columns
location_columns = ["location", 
                    "latitude", 
                    "longitude", 
                    "borough", 
                    "zip_code", 
                    "on_street_name", 
                    "off_street_name", 
                    "cross_street_name"]

# list the numeric columns
numeric_columns = ["number_of_persons_injured", 
                   "number_of_persons_killed", 
                   "number_of_pedestrians_injured", 
                   "number_of_pedestrians_killed", 
                   "number_of_cyclist_injured", 
                   "number_of_cyclist_killed", 
                   "number_of_motorist_injured", 
                   "number_of_motorist_killed"]"""

In [90]:
"""location_df = pd.DataFrame(data=df[location_columns], copy=True)
#location_df = df[location_columns]
numeric_df = df[numeric_columns]"""

In [ ]:
"""# https://stackoverflow.com/q/35491274"""

#### Extract coordinates from the location column

In [91]:
"""# first convert location column data from string to lists of strings

# function to convert a location string to a list
parse_location = lambda x: str(x).strip("()").split(", ")

#location_df.loc[:,"location"] = location_df.location.apply(parse_location)
location_df["location"] = location_df.location.apply(parse_location)"""

In [95]:
"""# then use the lists as the datasource for two extra colums
location_df[["latitude_check", "longitude_check"]] = pd.DataFrame(location_df.location.to_list())"""

In [97]:
"""# alternative aproach to the above: use regex to extract the coordinates
import re"""

In [119]:
"""df.location.fillna("").apply(lambda x: re.findall("[-]*\d+[.]\d+", x)).to_list()
# needs further work to split in lat & long"""

[['40.536728', '-74.193344'],
 ['40.829052', '-73.85038'],
 ['40.677303', '-73.804565'],
 ['40.624958', '-74.145775'],
 ['40.828846', '-73.90312'],
 ['40.556454', '-74.20777'],
 ['40.740025', '-73.97626'],
 ['40.651646', '-73.93233'],
 ['40.7518', '-73.817314'],
 ['40.816864', '-73.882744'],
 ['40.639614', '-74.08727'],
 ['40.71423', '-73.85386'],
 ['40.602757', '-73.96377'],
 ['40.652973', '-73.94401'],
 ['40.61074', '-73.92241'],
 ['40.69168', '-73.999344'],
 ['40.886536', '-73.89985'],
 ['40.686234', '-73.82418'],
 ['40.859875', '-73.89323'],
 ['40.848255', '-73.90976'],
 ['40.835358', '-73.94022'],
 ['40.727753', '-73.85861'],
 ['40.666702', '-73.76731'],
 ['40.66322', '-73.893654'],
 [],
 [],
 [],
 [],
 ['40.659714', '-73.739815'],
 ['40.86507', '-73.87185'],
 ['40.558887', '-74.1977'],
 ['40.678894', '-73.90347'],
 ['40.676373', '-73.84701'],
 ['40.84487', '-73.92068'],
 ['40.599243', '-73.752495'],
 ['40.8047', '-73.83141'],
 [],
 ['40.763287', '-73.80358'],
 ['40.651733', '-73.

In [123]:
"""# now verify if they match with the latitude and longitude columns
#str(location_df.lat) == str(location_df.latitude)
location_df"""

,location,latitude,longitude,borough,zip_code,on_street_name,off_street_name,cross_street_name,lat,lon,latitude_check,longitude_check
0,"[40.536728, -74.193344]",40.536728,-74.193344,STATEN ISLAND,10312.0,NaN,NaN,243 DARLINGTON AVENUE,40.536728,-74.193344,40.536728,-74.193344
1,"[40.829052, -73.85038]",40.829052,-73.850380,BRONX,10472.0,CASTLE HILL AVENUE,BLACKROCK AVENUE,NaN,40.829052,-73.85038,40.829052,-73.85038
2,"[40.677303, -73.804565]",40.677303,-73.804565,QUEENS,11420.0,135 STREET,FOCH BOULEVARD,NaN,40.677303,-73.804565,40.677303,-73.804565
3,"[40.624958, -74.145775]",40.624958,-74.145775,NaN,NaN,FOREST AVENUE,RICHMOND AVENUE,NaN,40.624958,-74.145775,40.624958,-74.145775
4,"[40.828846, -73.90312]",40.828846,-73.903120,BRONX,10456.0,NaN,NaN,1167 BOSTON ROAD,40.828846,-73.90312,40.828846,-73.90312
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,"[40.618893, -73.94642]",40.618893,-73.946420,BROOKLYN,11210.0,NaN,NaN,1314 EAST 29 STREET,40.618893,-73.94642,40.618893,-73.94642
99996,"[40.72338, -73.81475]",40.723380,-73.814750,QUEENS,11367.0,NaN,NaN,150-62 76 ROAD,40.72338,-73.81475,40.72338,-73.81475
99997,"[40.71082, -73.96853]",40.710820,-73.968530,BROOKLYN,11249.0,BROADWAY,KENT AVENUE,NaN,40.71082,-73.96853,40.71082,-73.96853
99998,"[40.63118, -73.928185]",40.631180,-73.928185,BROOKLYN,11234.0,NaN,NaN,1695 UTICA AVENUE,40.63118,-73.928185,40.63118,-73.928185


In [86]:
"""location_df.location[0]"""

['40.536728', '-74.193344']

In [65]:
"""loc["location"].map(lambda x : x.strip("()"))"""

AttributeError: 'float' object has no attribute 'strip'

In [56]:
"""float(df[location_columns].iloc[0].location.strip("()").split(", ")[1]) == df[location_columns].iloc[0].longitude"""

True

In [4]:
"""df.describe().T"""

,count,mean,std,min,25%,50%,75%,max
zip_code,64966.0,1.090165e+04,523.494905,10000.00000,1.045700e+04,1.120900e+04,1.135400e+04,1.169700e+04
latitude,91965.0,4.065192e+01,1.746143,0.00000,4.066792e+01,4.071772e+01,4.078560e+01,4.091217e+01
longitude,91965.0,-7.378199e+01,3.276307,-201.23706,-7.396087e+01,-7.391811e+01,-7.386286e+01,0.000000e+00
number_of_persons_injured,100000.0,3.719600e-01,0.743916,0.00000,0.000000e+00,0.000000e+00,1.000000e+00,1.500000e+01
number_of_persons_killed,100000.0,1.930000e-03,0.046112,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
number_of_pedestrians_injured,100000.0,4.739000e-02,0.223438,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00
number_of_pedestrians_killed,100000.0,6.400000e-04,0.025290,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
number_of_cyclist_injured,100000.0,4.964000e-02,0.222343,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
number_of_cyclist_killed,100000.0,2.500000e-04,0.015809,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
number_of_motorist_injured,100000.0,2.749200e-01,0.711058,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,1.500000e+01


In [5]:
"""profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)"""

In [6]:
"""profile.to_widgets()"""

Summarize dataset:   0%|          | 0/42 [00:00<?, ?it/s]

c:\users\philippe\appdata\local\programs\python\python39\lib\site-packages\scipy\stats\stats.py:4594: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
c:\users\philippe\appdata\local\programs\python\python39\lib\site-packages\scipy\stats\stats.py:4596: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(var) / np.sqrt(2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Other modifications

## Data export

In [15]:
df.to_csv("./data/data_first.csv")

## Next steps

*To be determined*